In [1]:
# The intent of the notebook is to begin modelling the baseball data. This will likely not be the final model, 
#     however,z it will act as a building block to more advanced models.
# Version: 1.0

In [5]:
import import_ipynb
from sklearn import svm
from pymysql import connect
from sys import path
from pathlib import Path
path.append('../../../') 
from BaseballAnalytics.bin.app_utils.queries import Queries
import numpy

In [6]:
# Connect to the database.
conn = connect(host="localhost", user="root", passwd="praquplDop#odlg73h?c", db="baseball_stats_db")
qu = Queries(conn)

In [127]:
# Get all the game ids. The information is returned as 
#    (Game_ID, year, day, month, Home_Score, Vis_Score, Home_Team, Visitng_Team, Home_Win) where a winning home team is flagged at 0.
game_outcomes = qu.get_game_outcomes()         
game_outcomes[0:5]

(('BOS199004090', 1990, 9, 4, 5, 2, 'BOS', 'DET', 0),
 ('CAL199004090', 1990, 9, 4, 4, 7, 'CAL', 'SEA', 1),
 ('CHA199004090', 1990, 9, 4, 2, 1, 'CHA', 'MIL', 0),
 ('HOU199004090', 1990, 9, 4, 4, 8, 'HOU', 'CIN', 1),
 ('KCA199004090', 1990, 9, 4, 6, 7, 'KCA', 'BAL', 1))

In [4]:
# Acquire only the outcomes.
game_only_outcomes = qu.get_only_outcomes()
game_only_outcomes[0:5]

[0, 1, 0, 1, 1]

In [5]:
# Retrieve only the game ids.
game_ids = [game_id[0] for game_id in game_outcomes]
print("The number of game ids: {}".format(len(game_ids)))
game_ids[0:5]

The number of game ids: 70101


['BOS199004090',
 'CAL199004090',
 'CHA199004090',
 'HOU199004090',
 'KCA199004090']

In [152]:
from warnings import filterwarnings
from BaseballAnalytics.bin.app_utils.common_help import Log_Helper
import pandas as pd
import pickle 

def fetch_data(query):

    # Function Description: Retrieve all the data given a particular query.
    # Function Parameters: query (The query to execute in the database.)
    # Function Throws: Nothing
    # Function Returns: The contents from the query.
    
    filterwarnings('error')    
    try:
        with conn.cursor() as c:
            c.execute(query)
            filterwarnings('always')
            return c.fetchall()
    except Exception as ex:
        print("The expections {}".format(ex))
        print("The query {}".format(query))
        raise Exception("A fire is buring in fetch_data.")

def convert_query_to_dict(data):
    
    # Function Description: Convert the data from a query into a dictionay to be indexed.
    #    The game id MUST BE the first value in the column.
    # Function Parameters: data (The data that was extractred from the query.)
    # Function Throws: Nothing
    # Function Returns: The dictionary containing a list of events associated with the Game Id
    
    query_dict = {}
    for row in data:
        if row[0] not in query_dict:
            query_dict[row[0]] = [list(row[1:])]
        else:
            game_list = query_dict[row[0]]
            game_list.append(list(row[1:]))
    return query_dict

def get_starting_batters(batting_players, game_id, team_batting):

    # Function Description: Givin all the batting events in a game, extract the starting the lineup.
    # Function Parameters: batting_players (The dictionary containing all the events from all games.), 
    #    game_id (The game id you wish to extract the betting lineup.), team_batting (The batting team: The Visting Team is 0, the Home Team is 1.)
    # Function Throws: Nothing
    # Function Returns: The list of players in the batting lineup.
                                                       
    filter_team = [(g_id, team, e_id) for g_id, team, e_id in batting_players[game_id] if team == team_batting] 
    filter_team.sort(key=lambda x: int(search(r'\d+', x[2][:3]).group()))      # Sort the string by the leading numbers of each id.
    player_count = 0                                                           # Add the first unique 9 players found in the event lineup.
    player_lineup = []
    for tup in filter_team:
        if tup[0] not in player_lineup: 
            player_lineup.append(tup[0])
            player_count += 1
        if player_count >= 9: break                     # Exit the loop when you have all the players.
    return player_lineup

def get_batters_in_all_games_vOne(prev_query=None):

    # Function Description: The function returns only the players and the starting pitcher with the respective game ids. This will be the first attempt of gathering
    #    the starting lineup.
    # Function Parameters: prev_query (The path to the results of a previous query.)
    # Function Throws: Nothing
    # Function Returns: A tuple containing two lists. The first list contains the home team names while the second list contains the away teams.

    # Home Team equals 1 for Batting Team. The query is formatted like such: 
    #     Game_ID, Batter_Name, Batting_Team, idEvent
    
    if prev_query != None:                  # Check if the query was executed before prior to performing another query.
        with open(prev_query, 'rb') as f:
            data = pickle.load(f)
            return convert_query_to_dict(data)
    game_participants = fetch_data("""select event_instance.Game_ID, batter_in_event.Batter_Name, 
                                      batter_in_event.Batting_Team, batter_in_event.idEvent 
	                                  from batter_in_event 
	                                  inner join event_instance on batter_in_event.idEvent=event_instance.idEvent
                                  """)
    with open(r'C:\Users\micha\Documents\Baseball_Analytics_Source_Data\model_v1\game_players.pickle', 'wb') as f:
        pickle.dump(game_participants, f)
    return convert_query_to_dict(game_participants)

In [124]:
def get_starting_pitcher(pitchers, game_id, team_batting):

    # Function Description: Givin all the batting events in a game, extract the starting the lineup.
    # Function Parameters: batting_players (The dictionary containing all the events from all games.), 
    #    game_id (The game id you wish to extract the betting lineup.), team_batting (The batting team: The Visting Team is 0, the Home Team is 1.)
    # Function Throws: Nothing
    # Function Returns: The list of players in the batting lineup.

    isolate_pitchers = [(team, player_id, e_id) for team, player_id, e_id in pitchers[game_id] if team == team_batting]     # Return only the first name for now.
    isolate_pitchers.sort(key=lambda x: int(search(r'\d+', x[2][:3]).group()))
    return isolate_pitchers[0][1]

def get_pitchers_in_all_games_vOne(prev_query=None):

    # Function Description: Retrieve the list of all pitchers who participated in every game.
    # Function Parameters: prev_query (The path to a previous query.)
    # Function Throws: Nothing
    # Function Returns: A dictionary with the game ids as keys storing the pitchers who participates.

    # Home Team equals 1 for Batting Team. The query is formatted like such: 
    #     Game_ID, Batter_Name, Batting_Team, idEvent
    
    if prev_query != None:                  # Check if the query was executed before prior to performing another query.
        with open(prev_query, 'rb') as f:
            data = pickle.load(f)
            return convert_query_to_dict(data)
    game_participants = fetch_data("""select DISTINCT event_instance.Game_ID, batter_in_event.Batting_Team, 
                                      pitcher_in_event.Pitcher_Name, pitcher_in_event.idEvent
                                         from event_instance
                                         inner join pitcher_in_event on pitcher_in_event.idEvent=event_instance.idEvent
                                         inner join batter_in_event on batter_in_event.idEvent=event_instance.idEvent
                                  """)
    with open(r'C:\Users\micha\Documents\Baseball_Analytics_Source_Data\model_v1\game_pitchers.pickle', 'wb') as f:
        pickle.dump(game_participants, f)
    return convert_query_to_dict(game_participants)

# def get_offensive_features(player_id, game_id):

#     # Function Description: Retrieve the features of a given player prior to entering the new game. The features are available from the previous game.
#     # Function Parameters: player_id (The player id associated in which we wish to retrieve the data.), game_id (The game id needed to look backwards.)
#     # Function Throws: Nothing
#     # Function Returns: A list containing the offensive features. The amount of features was determined in previous queries but does not matter in this function.
#     #    If the player does not have much data, I will be returning -1 to signal the prescence of a new player.

#     features = fetch_data("""
#                                 select Ten_Rolling_BA, Ten_Rolling_OBP, Ten_Rolling_SLG from offensive_features inner join
#                                 game_day on game_day.Game_ID=offensive_features.Game_ID
#                                 where player_id = '{}'
#                                 and game_day.Date < (select game_day.Date from game_day where game_day.Game_ID = '{}')
#                                 order by game_day.Date Desc;
#                                 """.format(player_id, game_id))
#     if len(features) < 10:
#         return [-1, -1, -1]
#     return list(features[0])                           # Return the first row which contains the data from the previous day.

# def get_pitchers_features(player_id, game_id):

#     # Function Description: Retrieve the features of a given pitcher prior to entering the new game. The features are available from the previous game.
#     # Function Parameters: player_id (The player id associated in which we wish to retrieve the data.), game_id (The game id needed to look backwards.)
#     # Function Throws: Nothing
#     # Function Returns: A list containing the offensive features. The amount of features was determined in previous queries but does not matter in this function.
#     #    If the player does not have much data, I will be returning -1 to signal the prescence of a new player.

#     features = fetch_data("""
#                                 select Ten_Rolling_Ks, Ten_Rolling_WHIP, Ten_Rolling_RA from pitching_features inner join
#                                 game_day on game_day.Game_ID=pitching_features.Game_ID
#                                 where player_id = '{}'
#                                 and game_day.Date < (select game_day.Date from game_day where game_day.Game_ID = '{}')
#                                 order by game_day.Date Desc;
#                                 """.format(player_id, game_id))
#     if len(features) < 5:                              # I reduced the number of previous games for pitcher by half.
#         return [-1, -1, -1]
#     return list(features[0])                           # Return the first row which contains the data from the previous day.

# def sub_pitching_features(pitchers, game_id):

#     # Function Description: Substitute the pitcher ids provided with the pitching features. The features will be returned in the same order the names are provided.
#     # Function Parameters: pitchers (The player id we wish to retrieve the data for.), game_id (The game id needed to look backwards.) 
#     # Function Throws: Nothing
#     # Function Returns: A complete list of the featues to be inputted into the model. The list will vary depending on the number of players and features for each player.

#     pitcher_features = []
#     print(len(pitchers))
#     for pitcher in pitchers:
#         pitcher_features += get_pitchers_features(pitcher, game_id)
#     return [float(feat) for feat in pitcher_features]

# def sub_offensive_features(batters, game_id):

#     # Function Description: Substitute the batters ids provided with the batting features. The features will be returned in the same order the names are provided.
#     # Function Parameters: batters (The player id we wish to retrieve the data for.), game_id (The game id needed to look backwards.) 
#     # Function Throws: Nothing
#     # Function Returns: A complete list of the featues to be inputted into the model. The list will vary depending on the number of players and features for each player.

#     offensive_features = []
#     for batter in batters:
#         print(batter)
#         offensive_features += get_offensive_features(batter, game_id)
#     print(type(offensive_features[0]))
#     return [float(feat) for feat in offensive_features]

# def get_game_features(game_id):

#     # Function Description: Get all the features for a given game id. This involves getting the players who played in the game and then retrieving their associated features.
#     # Function Parameters: game_id (The game id used to acquire the player features.)
#     # Function Throws: Nothing
#     # Function Returns: A single list containing the features of the game.

#     game_features = []
#     vis_players, home_players = get_players_in_game_vOne(game_id)
#     game_features += sub_pitching_features([vis_players[0]], game_id)         # Add all the visitor players to the feature sets.
#     game_features += sub_offensive_features(vis_players[1:], game_id)
#     game_features += sub_pitching_features([home_players[0]], game_id)        # Add all the home players to the feature sets.
#     game_features += sub_offensive_features(home_players[1:], game_id)             
#     print("At the night show")
#     return game_features

# def get_all_game_features(game_ids):

#     # Function Description: Given a list of game ids, retrieve the features for every game.
#     # Function Parameters: game_ids (The list of game ids.)
#     # Function Throws: Nothing
#     # Function Returns: A list of lists containing the game features.

#     num_games = len(game_ids)
#     all_game_features = {}
#     lh = Log_Helper()
#     lh.print_progress_bar(0, num_games, prefix = 'Progress:', suffix = 'Complete', length = 50)           # Initial call to print 0% progress
#     for num, game_id in enumerate(game_ids):
#         all_game_features[game_id] = get_game_features(game_id)
#         lh.print_progress_bar(num + 1, num_games, prefix = 'Progress:', suffix = 'Complete', length = 50)           # Initial call to print 0% progress
#     return all_game_features

In [125]:
# Fetch the players and pitchers from the game.
data_locs = Path(r'C:\Users\micha\Documents\Baseball_Analytics_Source_Data\model_v1')
pitchers_pickle = 'game_pitchers.pickle'
batters_pickle = 'game_players.pickle'
all_pitcher = get_pitchers_in_all_games_vOne(data_locs / pitchers_pickle)
all_batters = get_batters_in_all_games_vOne(data_locs / batters_pickle)

In [153]:
for game_id in game_outcomes:
    home_players = [get_starting_pitcher(all_pitcher, game_id[0], 0)]       # We want the pitchers in the event facing the Visting Batting Team. 
    home_players += get_starting_batters(all_batters, game_id[0], 1)
    vis_players = [get_starting_pitcher(all_pitcher, game_id[0], 1)]        # Vice versa.
    vis_players += get_starting_batters(all_batters, game_id[0], 0)
    break   ## Add the outcomes!!    